# Exercise 2: Temporal Hello World — Solution

Complete implementation of the Temporal Hello World workflow with an activity.

## Architecture Pattern 🏗️

```
Workflow Execution Request 👤
    ↓
Temporal Workflow (orchestration) 🎭
    ↓
Temporal Activity (unit of work) ⚙️
    ↓
[Process data, call APIs, etc.]
    ↓
Activity completes ✅
    ↓
Workflow returns result 📦
    ↓
Return to caller ✅
```

**Key Benefits:**
- ✅ Activities automatically retry on failure
- ✅ Workflow state persists across crashes
- ✅ Full execution history in Temporal UI

## Setup

Before running this solution, ensure you have:

1. **Temporal Server Running:**
   - Open `temporal_installation.ipynb` in the project root
   - Run all cells to install Temporal CLI and start the dev server
   - **Verify:** In Codespaces, go to the **Ports** tab → Find port **8233** → Click the **Globe icon** 🌐

2. **Dependencies Installed:**
   - Required packages should already be installed from the main setup


In [1]:
import asyncio
import nest_asyncio
from datetime import timedelta, datetime
import pytz
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker, UnsandboxedWorkflowRunner

## 1. Define an Activity

In [2]:
@activity.defn
async def process_data(data: str) -> str:
    """Activity that simulates work by upper-casing the payload."""
    activity.logger.info("🔧 Activity started")
    await asyncio.sleep(5)  # Simulate some processing delay
    result = f"Processed: {data.upper()}"
    activity.logger.info("✅ Activity completed")
    return result


## 2. Define a workflow

In [4]:
@workflow.defn
class HelloWorkflowTemporal:
    """Workflow that orchestrates the activity call."""

    @workflow.run
    async def run(self, name: str) -> str:
        workflow.logger.info("🚀 Workflow started")
        result = await workflow.execute_activity(
            process_data,
            args=[f"Hello {name}"],
            start_to_close_timeout=timedelta(seconds=10),
        )
        workflow.logger.info("✅ Workflow finished")
        return f"Workflow result: {result}"

## Create worker.py

In [ ]:
async def run_worker():  # Define async function to start and run the worker
    """Start a Temporal worker that listens for workflow and activity tasks."""
    # Connect to local Temporal server
    client = await Client.connect(
        "localhost:7233",  # Temporal server address
    )

    # Create worker that polls the task queue for work
    task_queue = "hello-temporal-task-queue"
    worker = Worker(
        client,  # Use the connected Temporal client
        task_queue=task_queue   ,  # Which queue to poll for tasks
        workflows=[HelloWorkflowTemporal],  # sets the workflow type name in UI
        activities=[process_data],
        workflow_runner=UnsandboxedWorkflowRunner(),  # List of activities this worker can execute
    )

    print(f"✅ Worker started on task queue: {task_queue}")
    print("   Listening for workflow and activity tasks...")
    # Start polling and executing tasks (blocks until stopped)
    await worker.run()


# Apply nest_asyncio to allow nested event loops in Jupyter
# Allow async functions to be called from within other async contexts
# This is needed because Jupyter notebooks already run in an event loop
nest_asyncio.apply()
# Create background task to run worker without blocking the notebook
# Create a task that runs the worker concurrently with other notebook operations
worker_task = asyncio.create_task(run_worker())
print("🔄 Worker running in background")


🔄 Worker running in background


✅ Worker started on task queue: hello-temporal-task-queue
   Listening for workflow and activity tasks...


## Create `starter.py`

Use the cell below to run the fully implemented solution script.

In [ ]:
async def run_solution():  # Define async function to execute the workflow
    """Execute Temporal workflow."""

    # Generate workflow ID with EST timestamp for human-readable tracking
    est = pytz.timezone("US/Eastern")  # Create EST timezone object
    now = datetime.now(est)  # Get current time in EST
    # Format timestamp as readable string with day-month-date-time pattern
    workflow_id = f"weather-{now.strftime('%a-%b-%d-%I%M%S').lower()}est"

    # Connect to Temporal server with OpenAI Agents SDK plugin
    client = await Client.connect(
        "localhost:7233"  # Enable OpenAI Agents integration
    )

    print(f"🚀 Starting workflow: {workflow_id}")

    # Start the workflow (non-blocking) and get handle for tracking
    task_queue = "hello-temporal-task-queue"
    handle = await client.start_workflow(
        HelloWorkflowTemporal.run,  # Workflow method to execute
        id=workflow_id,  # Unique workflow ID for tracking in Temporal UI
        task_queue=task_queue,  # Queue where worker will pick up this workflow
    )

    print(f"✅ Workflow started: {handle.id}")
    # Print Temporal UI link for observing workflow execution
    print(
        f"🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/{workflow_id}\n"
    )
    # Wait for workflow to complete and get result (blocking)
    result = await handle.result()
    print(f"🎉 Workflow completed with result: {result}")

# Run the solution with Jupyter-specific async handling
try:
    # Try to get existing event loop (Jupyter has one running)
    loop = asyncio.get_running_loop()
    # Execute in existing loop
    await run_solution()
except RuntimeError:
    # If no loop exists, create new one and run
    asyncio.run(run_solution())


Failed activation on workflow HelloWorkflowTemporal with ID weather-thu-nov-13-100328est and run ID 019a7dbe-32d5-7946-ace3-7009517ff00f
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py", line 418, in activate
    self._run_once(check_conditions=index == 1 or index == 2)
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py", line 2176, in _run_once
    raise self._current_activation_error
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py", line 2194, in _run_top_level_workflow_function
    await coro
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py", line 986, in run_workflow
    result = await self._inbound.execute_workflow(input)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workf

🚀 Starting workflow: weather-thu-nov-13-100328est
✅ Workflow started: weather-thu-nov-13-100328est
🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/weather-thu-nov-13-100328est

2025-11-13T15:03:28.230667Z  WARN temporal_sdk_core::worker::workflow: Failing workflow task run_id=019a7dbe-32d5-7946-ace3-7009517ff00f failure=Failure { failure: Some(Failure { message: "HelloWorkflowTemporal.run() missing 1 required positional argument: 'name'", source: "", stack_trace: "  File \"/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py\", line 418, in activate\n    self._run_once(check_conditions=index == 1 or index == 2)\n\n  File \"/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py\", line 2176, in _run_once\n    raise self._current_activation_error\n\n  File \"/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py\", line 2194, in _run_top_level_workflow_functio

Failed activation on workflow HelloWorkflowTemporal with ID weather-thu-nov-13-100328est and run ID 019a7dbe-32d5-7946-ace3-7009517ff00f
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py", line 418, in activate
    self._run_once(check_conditions=index == 1 or index == 2)
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py", line 2176, in _run_once
    raise self._current_activation_error
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py", line 2194, in _run_top_level_workflow_function
    await coro
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workflow_instance.py", line 986, in run_workflow
    result = await self._inbound.execute_workflow(input)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/worker/_workf

## Next Steps

**Ready for more?** Proceed to:

- **[Solution 3](../../solutions/03_durable_agent/solution.ipynb)** - Learn Temporal workflows and activities
- **[Solution 4](../../solutions/04_agent_routing/README.md)** - Combine agents + Temporal for production durability!